For the March 2024 Sky 1 and 2 datasets

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('result_sky1.csv')

In [ ]:
# Ensure the "Date" column is in datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%dT%H:%M:%S.%f")

# Function to correct a band's DataFrame
def correct_band_dataframe(df, band, t):
    # Find the reference date for this band
    object_count_per_date = df.groupby("Date")["Object"].nunique()
    reference_date = object_count_per_date.idxmax()
    
    # Initialize a DataFrame to hold the average magnitude differences
    avg_mag_diff = pd.DataFrame(columns=["Date", "Avg_Magnitude_Difference"])
    
    # Loop through each unique date in this band's DataFrame
    unique_dates = df["Date"].unique()
    for date in unique_dates:
        if date == reference_date:
            continue  # Skip the reference date itself

        airmass = df[df["Date"] == reference_date]["Airmass"].values[0]

        # Find common objects between the reference date and the current date
        ref_mag_col = f"Apparent magnitude {band} (instrumental)"
        common_stars = df[df["Date"] == reference_date][["Object", ref_mag_col]].merge(
            df[df["Date"] == date][["Object", ref_mag_col]], on="Object", suffixes=("_ref", "_other")
        )
        
        if common_stars.empty:
            # If no common stars, set average difference to zero
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Airmass": airmass,
                "Avg_Magnitude_Difference": 0.0
            }, ignore_index=True)
        else:
            # Calculate the average magnitude difference
            mag_difference = common_stars[f"{ref_mag_col}_ref"] - common_stars[f"{ref_mag_col}_other"]
            avg_diff = mag_difference.mean()
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Airmass": airmass,
                "Avg_Magnitude_Difference": avg_diff
            }, ignore_index=True)

    # Correct magnitudes for the current band
    new_mag_col = f"Corrected magnitude {band} (instrumental, {t})"
    for date in unique_dates:
        if date == reference_date:
            continue
        
        original_mag_col = f"Apparent magnitude {band} (instrumental)"
        row = avg_mag_diff[avg_mag_diff["Date"] == date]
        
        avg_diff = 0.0
        if not row.empty:
            avg_diff = row["Avg_Magnitude_Difference"].values[0]
        
        df.loc[df["Date"] == date, new_mag_col] = df.loc[df["Date"] == date, original_mag_col] + avg_diff
    # df[f"Apparent magnitude {band} (instrumental, {t})"] = df[f"Apparent magnitude {band} (instrumental)"]
    df[df["Date"] == reference_date][f"Corrected magnitude {band} (instrumental)"] = df[df["Date"] == reference_date][f"Apparent magnitude {band} (instrumental)"]
    df.drop(f"Apparent magnitude {band} (instrumental)", axis=1, inplace=True)
    # df[f'Airmass, {t}'] = df['Airmass']
    df.drop('Airmass', axis=1, inplace=True)
    return df

# Separate DataFrames by "Type"
types = data["Type"].unique()
type_dfs = {t: data[data["Type"] == t] for t in types}

# Correct each band's DataFrame for each type
corrected_band_dfs = {}

# Iterate by type first, then by band
for t in types:
    type_band_df = pd.DataFrame()
    
    if t == "ZWO":
        bands = ["R", "G", "B"]
    else:
        bands = ["R", "V", "B"]
    for band in bands:
        type_specific_band_df = type_dfs[t][["Object", "Date", "Airmass", f"Apparent magnitude {band} (instrumental)"]]
        # Drop rows with missing values in the apparent magnitude of that band
        type_specific_band_df = type_specific_band_df.dropna(subset=[f"Apparent magnitude {band} (instrumental)"])

        # Correct and concatenate results for this type and band
        type_specific_band_df = correct_band_dataframe(type_specific_band_df, band, t)

        type_band_df = pd.concat([type_band_df, type_specific_band_df])

    corrected_band_dfs[t] = type_band_df

# Join the corrected DataFrames based on "Date" and "Object"
joined_df = pd.DataFrame()

for t in types:
    if joined_df.empty:
        joined_df = corrected_band_dfs[t]
    else:
        joined_df = joined_df.merge(corrected_band_dfs[t], on=["Object", "Date"], how="outer")

In [ ]:
# # Group by 'Object'
# grouped = joined_df.groupby('Object')

# # Calculate the mean for all numeric columns
# mean_df = grouped.mean()

In [ ]:
# mean_df['Colour_B_G'] = mean_df['Adjusted magnitude B (instrumental, Bader)'] - mean_df['Adjusted magnitude V (instrumental, Bader)']
# mean_df['Colour_G_R'] = mean_df['Adjusted magnitude V (instrumental, Bader)'] - mean_df['Adjusted magnitude R (instrumental, Bader)']

In [ ]:
# # Focus on ZWO data
# zwo_data = joined_df[
#     (joined_df["Adjusted magnitude R (instrumental, ZWO)"].notna()) |
#     (joined_df["Adjusted magnitude G (instrumental, ZWO)"].notna()) |
#     (joined_df["Adjusted magnitude B (instrumental, ZWO)"].notna())
# ]

# # Convert 'Date' to datetime
# zwo_data["Date"] = pd.to_datetime(zwo_data["Date"])

# # Define a function to find the closest row by time
# def find_closest(object_name, date, magnitude_column):
#     subset = zwo_data[zwo_data["Object"] == object_name]
#     subset = subset[subset[magnitude_column].notna()]
#     if subset.empty:
#         return np.nan
#     closest_row = subset.iloc[(subset["Date"] - date).abs().argsort()[:1]]
#     return closest_row[magnitude_column].values[0]

# # Define a function to calculate Colour_B_G based on which magnitude is available
# def calculate_colour_b_g(row):
#     if pd.notna(row["Adjusted magnitude B (instrumental, ZWO)"]):
#         # Calculate using "B"
#         b_value = row["Adjusted magnitude B (instrumental, ZWO)"]
#         g_value = find_closest(row["Object"], row["Date"], "Adjusted magnitude G (instrumental, ZWO)")
#         return b_value - g_value
#     elif pd.notna(row["Adjusted magnitude G (instrumental, ZWO)"]):
#         # Calculate using "G"
#         b_value = find_closest(row["Object"], row["Date"], "Adjusted magnitude B (instrumental, ZWO)")
#         g_value = row["Adjusted magnitude G (instrumental, ZWO)"]
#         return b_value - g_value
#     return np.nan

# # Apply the function to calculate Colour_B_G
# zwo_data["Colour_B_G"] = zwo_data.apply(calculate_colour_b_g, axis=1)

# # Calculate Colour_G_R
# zwo_data["Colour_G_R"] = zwo_data.apply(
#     lambda row: 
#         find_closest(row["Object"], row["Date"], "Adjusted magnitude G (instrumental, ZWO)") -
#         row["Adjusted magnitude R (instrumental, ZWO)"], 
#     axis=1
# )

In [ ]:
# mask = (
#     (zwo_data['Colour_B_G'].isna() & zwo_data['Colour_G_R'].isna())
# )

# zwo_data_ = zwo_data[~mask]

In [ ]:
# bader_data = joined_df[
#     (joined_df["Adjusted magnitude R (instrumental, Bader)"].notna()) |
#     (joined_df["Adjusted magnitude V (instrumental, Bader)"].notna()) |
#     (joined_df["Adjusted magnitude B (instrumental, Bader)"].notna())
# ]

In [ ]:
# # Function to find the closest Bader row by time with the same object
# def find_closest_bader(row, magnitude_column):
#     # Extract the corresponding subset of Bader data for the same object
#     subset = bader_data[bader_data["Object"] == row["Object"]]
    
#     # Ensure there's at least one valid value in the magnitude column
#     subset = subset[subset[magnitude_column].notna()]
#     if subset.empty:
#         return np.nan, None
    
#     # Find the row with the closest time
#     closest_row = subset.iloc[(subset["Date"] - row["Date"]).abs().argsort()[:1]]
    
#     return closest_row["Date"].values[0], closest_row[magnitude_column].values[0]

# # Add new columns to hold closest Bader information
# zwo_data["Closest Bader Date"] = None
# # zwo_data["Closest Bader Magnitude"] = None

# # Match each row in ZWO data with the closest corresponding row in Bader
# for index, row in zwo_data.iterrows():
#     if pd.notna(row["Adjusted magnitude R (instrumental, ZWO)"]):
#         closest_date, closest_magnitude = find_closest_bader(row, "Adjusted magnitude R (instrumental, Bader)")
#         band = "R"
#     elif pd.notna(row["Adjusted magnitude G (instrumental, ZWO)"]):
#         closest_date, closest_magnitude = find_closest_bader(row, "Adjusted magnitude V (instrumental, Bader)")
#         band = "V"
#     elif pd.notna(row["Adjusted magnitude B (instrumental, ZWO)"]):
#         closest_date, closest_magnitude = find_closest_bader(row, "Adjusted magnitude B (instrumental, Bader)")
#         band = "B"

#     zwo_data.at[index, "Closest Bader Date"] = closest_date
#     zwo_data.at[index, f"Closest Bader Magnitude, {band}"] = closest_magnitude

In [ ]:
# # Drop empty columns
# zwo_data.dropna(axis=1, how='all', inplace=True)

# # Remove rows where both Colour_B_G and Colour_G_R are NaN
# colour_columns = ["Colour_B_G", "Colour_G_R"]
# zwo_data.dropna(subset=colour_columns, how='all', inplace=True)

# # Remove rows without a corresponding Bader magnitude
# zwo_data.dropna(subset=["Closest Bader Date"], inplace=True)

In [ ]:
final_df = data.merge(joined_df, on=["Object", "Date"], how="outer")
final_df.to_csv('result_sky1_corr.csv', index=False)